In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('songdata.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [4]:
df.shape

(57650, 4)

In [5]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [6]:
df.shape

(5000, 3)

In [7]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [9]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached regex-2023.10.3-cp311-cp311-win_amd64.whl.metadata (41 kB)
Using cached regex-2023.10.3-cp311-cp311-win_amd64.whl (269 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\nltk.exe' -> 'C:\\Python311\\Scripts\\nltk.exe.deleteme'



In [11]:
df['text'][0]

"don't like the direction you are going to   seems to lack the attention, that it used to   stay out all night and get high with your friends   wonder why you don't get one thing done   don't like the direction you are going to   ahhh la da la da ahhh la da la da      don't like the direction you have come to   now it has the attention that it's used to   stay home all night with the tv and wife   comfortable life's not all it's cracked up to be   don't like the direction you have come to      it's easy to get caught and the weight of the world   is falling on your face, so unsure that you...   don't like the direction you are going to   seems to lack the attention that it's usted to   stay out all night and get high woth your friends   wonder why you don't get a one thing done   don't like the direction you are going to   ahhh ahhh      yeah it's easy to get caught and the weight of the world   is falling on your face, so unsure that you...   don't like like the direction you are goin

In [10]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [12]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [15]:
similarity[0]

array([1.        , 0.00524393, 0.02902698, ..., 0.04081673, 0.00591066,
       0.02023972])

# recommedation function

In [17]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [18]:
recommendation('Alma Mater')

['Say Goodbye',
 'Born Cross-eyed',
 'Everybody Knows This Is Nowhere',
 'Goodbye Little Columbus',
 'Kiss It Goodbye',
 'Long, Long Way To Go',
 'Out Of Goodbyes',
 'Everybody Loves Me',
 'All Of Me',
 'Men Are All The Same',
 'Falling In Love',
 'Back Together',
 'The Great Getaway',
 'Looking For You',
 '(Everybody) Get Down',
 'Lately',
 'Married With Children',
 "This Ain't Goodbye",
 'The Last Time',
 "I'll Still Be Missing You"]

In [19]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))